In [ ]:
# Package installations
# !pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Package imports

import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, concatenate, LSTM, Reshape, Lambda, GlobalAveragePooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import pandas as pd
import csv
import zipfile

In [ ]:
tf.debugging.disable_traceback_filtering()

In [ ]:
home_dir = '/content'
dataset_dir = f'{home_dir}/drive/MyDrive/Drift/Training/Data/dataset_004'
temp_images_dir = f'{home_dir}/temp_images'

In [ ]:
with zipfile.ZipFile(f'{dataset_dir}/images.zip', 'r') as zip_ref:
  zip_ref.extractall(temp_images_dir)

In [ ]:
class CustomMetrics(Callback):
    def __init__(self, val_generator, val_steps):
        super().__init__()
        self.val_generator = val_generator
        self.val_steps = val_steps

    def on_epoch_end(self, epoch, logs=None):
        val_labels = []
        val_predictions = []

        for i in range(self.val_steps):
            val_data, val_label = self.val_generator[i]
            val_pred = self.model.predict(val_data)
            val_labels.append(val_label)
            val_predictions.append(val_pred)

        val_labels = np.concatenate(val_labels)
        val_predictions = np.concatenate(val_predictions)

        # Ensure shapes match
        assert val_predictions.shape == val_labels.shape, f"Shape mismatch: {val_predictions.shape} vs {val_labels.shape}"

        # Compute mean absolute error
        mae = np.mean(np.abs(val_predictions - val_labels))

        # Compute R2 score
        r2 = r2_score(val_labels, val_predictions)

        # Detailed logging
        print(f'--- Epoch {epoch + 1} ---')
        print(f'Loss: {logs["loss"]:.4f}')
        print(f'Validation Loss: {logs["val_loss"]:.4f}')
        print(f'Validation MAE: {logs["val_mae"]:.4f}')
        print(f'Mean Absolute Error (MAE): {mae:.4f}')
        print(f'R2 Score: {r2:.4f}')
        print('---------------------')


In [ ]:
class DirectoryDataGenerator(Sequence):
    def __init__(self, image_files, y_controls, csv, batch_size, image_shape, num_classes=None, shuffle=True):
        self.y_controls = y_controls
        self.batch_size = batch_size
        self.image_shape = image_shape
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.file_list = image_files
        self.csv_data = csv
        self.on_epoch_end()

        print(f"Initialized DirectoryDataGenerator with {len(self.file_list)} images and {len(self.y_controls)} controls.")

    def __len__(self):
        return int(np.floor(len(self.file_list) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]
        X_images, y = self._data_generation(batch_files, index)
        X_features = self._extract_features(index * self.batch_size, len(batch_files))
        return [X_images, X_features], y

    def on_epoch_end(self):
        if self.shuffle:
            # Combine file_list and y_controls into a single list of tuples
            combined = list(zip(self.file_list, self.y_controls))

            # Shuffle the combined list
            np.random.shuffle(combined)

            # Unzip the shuffled list back into file_list and y_controls
            self.file_list, self.y_controls = zip(*combined)

            # Convert back to list (zip returns tuples)
            self.file_list = list(self.file_list)
            self.y_controls = np.array(self.y_controls)

            print(f"Epoch ended with {len(self.file_list)} images and {len(self.y_controls)} controls.")


    def _data_generation(self, batch_files, index):
        start_index = index * self.batch_size
        end_index = start_index + self.batch_size
        # print(f"Generation index: {index}")
        # print(f"start_index: {start_index}, end_index: {end_index}, y_controls length: {len(self.y_controls)}")

        # Adjust the end_index if it exceeds the length of y_controls
        if end_index > len(self.y_controls):
            end_index = len(self.y_controls)

        current_batch_size = end_index - start_index

        X = np.empty((self.batch_size, *self.image_shape))  # always create array with full batch size
        y = np.empty((self.batch_size, self.y_controls.shape[1]))  # always create array with full batch size

        actual_y = self.y_controls[start_index:end_index]
        # print(f"X shape: {X.shape}, y shape: {y.shape}")

        for i, file_name in enumerate(batch_files[:current_batch_size]):
            image = Image.open(file_name)
            image = image.resize((self.image_shape[1], self.image_shape[0]))
            image = np.array(image) / 255.0  # Normalize to [0, 1]

            X[i,] = image

        # Handle the case where the current batch is smaller than the batch size
        if current_batch_size < self.batch_size:
            X[current_batch_size:,] = 0  # pad the remaining batch with zeros or any other value
            y[current_batch_size:,] = 0  # pad the remaining batch with zeros or any other value

        y[:current_batch_size] = actual_y

        return X, y


    def _extract_features(self, start_index, batch_size):
        features = []
        for i in range(batch_size):
            feature_row = self.csv_data.iloc[start_index + i].values  # Use iloc for row extraction
            features.append(feature_row)
        return np.array(features)


In [ ]:
# Custom activation function to constrain the output
def custom_activation(x):
    # Apply sigmoid to throttle and brake to ensure [0, 1] range
    throttle = tf.keras.activations.sigmoid(x[:, 0])
    brake = tf.keras.activations.sigmoid(x[:, 2])

    # Apply tanh to steer to ensure [-1, 1] range
    steer = tf.keras.activations.tanh(x[:, 1])

    # Apply sigmoid and then round to get boolean values for hand_brake, reverse, manual_gear_shift
    hand_brake = tf.round(tf.keras.activations.sigmoid(x[:, 3]))
    reverse = tf.round(tf.keras.activations.sigmoid(x[:, 4]))
    manual_gear_shift = tf.round(tf.keras.activations.sigmoid(x[:, 5]))

    # Apply linear activation to gear and then round to get integer values
    gear = tf.round(x[:, 6])

    # Concatenate all the processed components back into a single tensor
    return tf.stack([throttle, steer, brake, hand_brake, reverse, manual_gear_shift, gear], axis=1)

In [ ]:
image_shape = (600, 800, 4)  # Adjust based on your image dimensions
y_controls = np.load(f'{dataset_dir}/y_controls.npy')  # Assuming y_controls is saved in a .npy file
csv_path = f'{dataset_dir}/measurements.csv'  # Path to CSV file with additional features
batch_size = 100  # Adjust batch size as needed
num_classes = 7  # Assuming 7 outputs, adjust based on your use case

print(f"y_controls shape: {y_controls.shape}")

y_controls shape: (11886, 7)


In [ ]:
# Extract the list of image files
file_list = sorted(
    [os.path.join(temp_images_dir + "/images", f) for f in os.listdir(temp_images_dir + "/images") if f.endswith('.png')],
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
)

print(f"Total number of files: {len(file_list)}")


Total number of files: 11886


In [ ]:
# Trim dataset

# Ensure the number of files and y_controls match after trimming
trim_amount = 1886

file_list = file_list[:-trim_amount] # trim dataset
y_controls = y_controls[:-trim_amount, :] # Trim dataset
print(f"trimmed y_controls shape: {y_controls.shape}")
print(f"trimmed file_list shape: {len(file_list)}")

trimmed y_controls shape: (10000, 7)
trimmed file_list shape: 10000


In [ ]:
measurements = pd.read_csv(csv_path, header=None)
print(f"measurements shape: {measurements.shape}")
measurements = measurements.iloc[:-trim_amount, :]

print(f"measurements shape: {measurements.shape}")

# # Ensure that all arrays have the same length
assert len(file_list) == len(y_controls) == len(measurements), "All arrays must have the same length"



measurements shape: (11886, 6)
measurements shape: (10000, 6)


In [ ]:
# Assuming the necessary imports and variable definitions are already done



# Verify file counts
print(f"Total number of files: {len(file_list)}")
print(f"Total number of labels: {len(y_controls)}")

# Split the data into training and validation sets
train_files, val_files, y_train, y_val, train_measurements, val_measurements = train_test_split(
    file_list, y_controls, measurements, test_size=0.1, random_state=42
)
# Verify shapes after splitting
print(f"Train row count: {len(train_files)}, Train labels shape: {y_train.shape}")
print(f"Validation row count: {len(val_files)}, Validation labels shape: {y_val.shape}")

# Create generators for training and validation
# TODO renable shuffle on training generator
train_generator = DirectoryDataGenerator(train_files, y_train, train_measurements, batch_size, image_shape, num_classes, shuffle=True)
val_generator = DirectoryDataGenerator(val_files, y_val, val_measurements, batch_size, image_shape, num_classes, shuffle=False)

Total number of files: 10000
Total number of labels: 10000
Train row count: 9000, Train labels shape: (9000, 7)
Validation row count: 1000, Validation labels shape: (1000, 7)
Epoch ended with 9000 images and 9000 controls.
Initialized DirectoryDataGenerator with 9000 images and 9000 controls.
Initialized DirectoryDataGenerator with 1000 images and 1000 controls.


In [ ]:
# Image input branch
image_input = Input(shape=(224, 224, 3), name='image_input')

x = Conv2D(32, (3, 3), padding='same')(image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)

# Additional features input branch
features_input = Input(shape=(6,), name='features_input')  # Adjust num_features based on your CSV data

# Combine the outputs of the two branches
combined = concatenate([x, features_input])

# Fully connected layers after concatenation
z = Dense(256, activation='relu')(combined)
z = Dense(128, activation='relu')(z)
z = Dense(64, activation='relu')(z)

# Output Layer
output = Dense(7, activation='linear')(z)  # Assuming 7 outputs

# Apply custom activation for range constraints
output = Lambda(custom_activation)(output)

# Define the model
model = Model(inputs=[image_input, features_input], outputs=output)

In [ ]:
# Compile the model
optimizer = AdamW(learning_rate=0.0001)
metrics = [
    'mae',
    tf.keras.metrics.RootMeanSquaredError(),
    # tf.keras.metrics.MeanAbsoluteError(),
    # tf.keras.metrics.MeanSquaredError(),
    # tf.keras.metrics.MeanAbsolutePercentageError(),
    # tf.keras.metrics.MeanSquaredLogarithmicError(),
    # tf.keras.metrics.CosineSimilarity(),
]
model.compile(optimizer=optimizer, loss='mse', metrics=metrics)

# Print the model summary
model.summary()

# Calculate the number of validation steps
val_steps = len(val_generator)

# Create an instance of the CustomMetrics callback with validation data
custom_metrics = CustomMetrics(val_generator, val_steps)

# Create an instance of the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using the generator and the custom callbacks
history = model.fit(train_generator, validation_data=val_generator, epochs=20, callbacks=[custom_metrics])

In [ ]:
# Save the model
model.save_weights('/content/drive/MyDrive/Drift/Training/Models/vehicle_control_model_015.h5')
